In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas_profiling as pp
from datetime import datetime
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
import eli5
from eli5.sklearn import PermutationImportance

import scikitplot as skplt
from sklearn.decomposition import PCA

sns.set_style('whitegrid')
pd.set_option('display.max_columns', None) # display all columns

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import glob

## Loading datasets
#### read dataset, define delimiter, dtypes and encoding

In [6]:
!ls -l ../data/dataset/A_DB_NRJ/

total 5602420
-rw-rw-r-- 1 jovyan 1000 325753335 Oct 24 09:30 a_20191003.csv
-rw-rw-r-- 1 jovyan 1000 325847319 Oct 24 09:30 a_20191004.csv
-rw-rw-r-- 1 jovyan 1000 204579499 Oct 24 09:31 a_20191005.csv
-rw-rw-r-- 1 jovyan 1000 156022232 Oct 24 09:31 a_20191006.csv
-rw-rw-r-- 1 jovyan 1000 345785736 Oct 24 09:31 a_20191007.csv
-rw-rw-r-- 1 jovyan 1000 411527423 Oct 24 09:32 a_20191008.csv
-rw-rw-r-- 1 jovyan 1000 397576102 Oct 24 09:32 a_20191009.csv
-rw-rw-r-- 1 jovyan 1000 398615603 Oct 24 09:33 a_20191010.csv
-rw-rw-r-- 1 jovyan 1000 387487250 Oct 24 09:34 a_20191011.csv
-rw-rw-r-- 1 jovyan 1000 250963109 Oct 24 09:34 a_20191012.csv
-rw-rw-r-- 1 jovyan 1000 190969537 Oct 24 09:34 a_20191013.csv
-rw-rw-r-- 1 jovyan 1000 374611350 Oct 24 09:35 a_20191014.csv
-rw-rw-r-- 1 jovyan 1000 390035144 Oct 24 09:35 a_20191015.csv
-rw-rw-r-- 1 jovyan 1000 393187960 Oct 24 09:36 a_20191016.csv
-rw-rw-r-- 1 jovyan 1000 411435843 Oct 24 09:36 a_20191017.csv
-rw-rw-r-- 1 jovyan 1000 367993973 Oct 24

In [11]:
# view top 5 lines from raw file
!head -n 5 ../data/dataset/A_DB_NRJ/a_20191003.csv

In [42]:
directory_names = ["A_DB_NRJ", "A_Logs_Podcasts", "A_NRJ_Premium", "Google_Analytics"]
for dirk in directory_names:
    path = r'../data/dataset/'+dirk # use your path
    all_files = glob.glob(path + "/*.csv")
    li = []

    for filename in all_files:
        df = pd.read_csv(filename)
        li.append(df)

    df = pd.concat(li, axis=0, ignore_index=True)
    pp.ProfileReport(df.sample(2500)).to_file('../data/report/'+dirk+'.html')

In [43]:
directory_names = ["Google_Analytics"]
for dirk in directory_names:
    path = r'../data/dataset/'+dirk # use your path
    all_files = glob.glob(path + "/*.csv")
    li = []

    for filename in all_files:
        df = pd.read_csv(filename)
        li.append(df)

    df = pd.concat(li, axis=0, ignore_index=True)
    pp.ProfileReport(df).to_file('../data/report/'+dirk+'.html')

In [51]:
#df = pd.read_csv('../data/dataset/Google_Analytics/GA_20191003.csv')
path = r'../data/dataset/Google_Analytics' # use your path
all_files = glob.glob(path + "/*.csv")
li = []
#print("li")

for filename in all_files:
    df = pd.read_csv(filename, delimiter = ',',
                     encoding='utf-8',
                     dtype={"ga:clientId":"str",
                            "ga:pagepath":"str",
                            "ga:dateHourMinute":"str",
                            "ga:latitude":"float",
                            "ga:longitude":"float",
                            "ga:sourceMedium":"str",
                            "ga:timeOnPage":"int"}
                            )
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [52]:
df.sample(4).T

,44088,211101,143467,137853
ga:clientId,155684311.1554183607,431901193.1571236062,1959366294.1570435,1144912801.156922
ga:pagepath,/index.html,/dreambooster,/player,/radioplayer/NRJhits2000
ga:dateHourMinute,201910031425,201910161627,201910070940,201910071026
ga:latitude,50.5187,50.8503,43.6047,50.4542
ga:longitude,5.2397,4.3517,1.4442,3.9567
ga:sourceMedium,(direct) / (none),m.facebook.com / referral,google / organic,google / organic
ga:timeOnPage,12,0,9,0


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295802 entries, 0 to 295801
Data columns (total 7 columns):
ga:clientId          295802 non-null object
ga:pagepath          295802 non-null object
ga:dateHourMinute    295802 non-null object
ga:latitude          295802 non-null float64
ga:longitude         295802 non-null float64
ga:sourceMedium      295802 non-null object
ga:timeOnPage        295802 non-null int64
dtypes: float64(2), int64(1), object(4)
memory usage: 15.8+ MB


In [56]:
pp.ProfileReport(df.sample(2500)).to_file('../data/report/Google_Analytic1.html')

In [55]:
# advanced way for df.describe()

pp.ProfileReport(df.sample(2500)).to_file('../data/report/Google_Analytic1.html')

In [ ]:
# df = pd.read_csv('../data/dataset/A_DB_NRJ/a_20191003.csv',
#                  delimiter = ',',
#                  encoding='utf-8',
#                  dtype={"id":"str",
#                         "game_id":"str"})

## Summarize data

In [39]:
# advanced way for df.describe()
# sampling 1% for a report
pp.ProfileReport(df.sample(1000)).to_file('../data/report/a_20191003.html')